In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16, 'figure.figsize': (40, 8), 'font.family': 'serif', 'text.usetex': True, 'pgf.rcfonts': False})

%load_ext autoreload
%autoreload 2

import jax
jax.config.update('jax_platform_name', 'cpu')
    
from jax_smolyak import indices
from jax_smolyak import points
import jax_smolyak.smolyak_jax as sj
import jax_smolyak.smolyak as sn

def target_f_1d(x):
    x = np.atleast_1d(x)
    y = np.sin(2 * np.pi * x) * np.cos(4 * np.pi * x)
    return y if x.shape[0] > 1 else y[0]

def show_interpolation(target, nodes, degree) :
    l = indices.find_suitable_l([1], n=degree+1, nested=nodes.is_nested)
    ip = sj.MultivariateSmolyakBarycentricInterpolator(g=nodes, k=[1], l=l, rank=1, f=target)
    ipn = sn.SmolyakBarycentricInterpolator(g=nodes, k=[1], l=l, f=target)
    
    xi = nodes[0](degree)
    if np.min(xi) != np.max(xi) :
        x = np.linspace(np.min(xi), np.max(xi), 200)
    else :
        x = np.linspace(-1, 1, 200)
    y_f = target(x)
    y_ip = ip(x[:, None])
    assert np.isnan(y_f).any() == np.isinf(y_f).any() == False
    
    fig, axes = plt.subplots(1, 2, figsize=(20, 5))    
    axes[0].plot(x, y_f, lw=2, label=r'target $f(x)$')
    axes[0].plot(x, y_ip, lw=2, label=r'interpolant jax $I[f](x)$')
    axes[0].plot(x, ipn(x[:, None]), lw=2, ls='--', label=r'interpolant numpy $I[f](x)$')
    axes[0].scatter(xi, target(xi), color='k')
    axes[1].plot(x, y_f - y_ip[:,0], lw=2, label='error $f(x)-I[f](x)$')
    axes[1].scatter(xi, np.zeros((len(xi),)), color='k')
    for ax in axes : ax.grid(); ax.legend(); ax.set_xlabel('x')

# Leja interpolation nodes

In [ ]:
show_interpolation(target_f_1d, points.LejaMulti(domains=[[-2.,1.]]), 10)

In [ ]:
show_interpolation(target_f_1d, points.LejaMulti(domains=[[-2.,1.]]), 35)

# Gauß-Hermite interpolation nodes

In [ ]:
show_interpolation(target_f_1d, points.GaussHermiteMulti(np.array([0]), np.array([.25])), 10)

In [ ]:
show_interpolation(target_f_1d, points.GaussHermiteMulti(np.array([0]), np.array([.13])), 30)